### Data Cleaning / Merging

In [1]:
# some imports
import pandas as pd
import numpy as np

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# load data (converted from json to csv using yelp git repo converter) (takes a few min)
# data structure: https://www.yelp.co.uk/dataset_challenge/dataset
business = pd.read_csv('yelp_academic_dataset_business.csv')
review = pd.read_csv('yelp_academic_dataset_review.csv')
#user = pd.read_csv('yelp_academic_dataset_user.csv')

#print len(business), len(review), len(user)

c:\Users\docmario\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (1,4,7,14,18,21,27,30,43,49,52,62,64,66,70,84,89,92,100) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# merge datasets
merged = pd.merge(review,business,on='business_id')
print(len(merged))
merged.head()

1569264


,user_id,review_id,text,votes.cool,business_id,votes.funny,stars_x,date,type_x,votes.useful,...,attributes.Smoking,attributes.Attire,attributes.Hair Types Specialized In.curly,attributes.Good For Groups,neighborhoods,attributes.Open 24 Hours,attributes.Ambience.romantic,attributes.Hair Types Specialized In.perms,attributes.Music.jukebox,attributes.Ambience.upscale
0,Xqd0DzHaiyRqVH3WRG7hzg,15SdjuK7DmYqUAj6rjGowg,dr. goldberg offers everything i look for in a...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,5,2007-05-17,review,2,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN
1,H1kH6QZV7Le4zqTRNxoZow,RF6UnRTtG7tWMcrO2GEoAg,"Unfortunately, the frustration of being Dr. Go...",0,vcNAWiLM4dR7D2nwwJ7nCA,0,2,2010-03-22,review,2,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN
2,zvJCcrpm2yOZrxKffwGQLA,-TsVN230RCkLYKBeLsuz7A,Dr. Goldberg has been my doctor for years and ...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,4,2012-02-14,review,1,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN
3,KBLW4wJA_fwoWmMhiHRVOA,dNocEAyUucjT371NNND41Q,Been going to Dr. Goldberg for over 10 years. ...,0,vcNAWiLM4dR7D2nwwJ7nCA,0,4,2012-03-02,review,0,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN
4,zvJCcrpm2yOZrxKffwGQLA,ebcN2aqmNUuYNoyvQErgnA,Got a letter in the mail last week that said D...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,4,2012-05-15,review,2,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN


In [37]:
subset = merged[['name', 'state', 'city', 'stars_y', 'categories', 'stars_x', 'text']]

In [ ]:
# understand features/columns
#print(len(merged.columns)
cols = [x for x in merged.columns.tolist() if 'attributes' in x]
#'attributes.Ambience.divey'[11:]
#merged.apply(lambda x: [i[11:] for i in cols if x[i]], axis=1)
def getAttributes(x):
    attrs = ""
    for i in cols:
        if type(x[i]) is bool and x[i]:
            attrs += ",%s" % i
        elif type(x[i]) is str:
            attrs += ",%s=%s" % (i, x[i])  
        elif not np.isnan(x[i]):
            attrs += ",%s" % i
            
    return attrs[1:] if attrs else attrs

#print(merged.iloc[:10].apply(getAttributes, axis=1)[0])
#merged.iloc[1251:1260].apply(lambda x: [type(x[i]) for i in cols], axis=1)[1251]
subset['attributes'] = merged.apply(getAttributes, axis=1)

In [ ]:
1

In [14]:
# clean text of special characters and leading/trailing spaces
clean['text'] = clean['text'].str.replace('\n', ' ').str.strip()
clean['text'] = clean['text'].str.replace('\f', ' ')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [15]:
clean.city.value_counts()

Las Vegas                    617358
Phoenix                      219815
Scottsdale                   120756
Charlotte                     86952
Tempe                         64978
Pittsburgh                    62325
Henderson                     49827
Chandler                      41003
Mesa                          38880
Madison                       37071
Montréal                      28683
Gilbert                       26797
Glendale                      23544
Edinburgh                     23458
Montreal                      16459
Peoria                        11379
North Las Vegas                9922
Champaign                      8495
Surprise                       7189
Goodyear                       6371
Avondale                       4683
Queen Creek                    3861
Cave Creek                     3635
Urbana                         3518
Matthews                       3241
Middleton                      2462
Karlsruhe                      2406
Fort Mill                   

In [16]:
# create a smaller subset for testing
Charlotte = clean[clean.city == 'Charlotte']

In [25]:
pd.set_option('max_colwidth', 800)
Charlotte['text']

891                                                                                                                                                                                                                                                                                                                                                                                            Good location for a business traveler. There are plenty of restaurant options nearby, and downtown isn't too far. The only thing is the traffic around this area gets a little clogged around rush hour, but I guess that's to be expected.  The mall is nearby, too, if you need some shopping.   Room was good--standard Courtyard. Not as many HD television channels as I would have hoped in 2014. Wireless internet was not terrible.
919        Great stuff.  They said they had limited menu selections, but there was a fixed menu option from which you could select an appetizer, entree and dessert.  Everything was deliciou

In [47]:
# pivot out categories column 
import ast

L = [] # empty list to store categories

for l in Charlotte.categories.unique():
    L.extend(ast.literal_eval(l)) # literal turns string list into list list

print len(set(L)), 'categories'
cat = list(set(L)) # store distinct list

487 categories


In [51]:
# for each category, pivot map
for c in cat:
    Charlotte[c] = Charlotte.categories.map(lambda x: c in x)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [52]:
Charlotte.head()

,user_id,review_id,text,business_id,stars_x,date,type_x,categories,name_x,attributes.Good For.dinner,...,Spanish,Windows Installation,Internal Medicine,Mexican,Car Dealers,Meat Shops,Traditional Chinese Medicine,Financial Services,Specialty Food,Optometrists
891,072tu2_MFWmw-jb1O6DQYg,F2OptIJ6bvYnUTIdOYy9oA,"Good location for a business traveler. There are plenty of restaurant options nearby, and downtown isn't too far. The only thing is the traffic around this area gets a little clogged around rush hour, but I guess that's to be expected. The mall is nearby, too, if you need some shopping. Room was good--standard Courtyard. Not as many HD television channels as I would have hoped in 2014. Wireless internet was not terrible.",k8KP9qHJeYW0-nU7ykLu9w,4,2014-03-24,review,"['Hotels & Travel', 'Event Planning & Services', 'Hotels']",Courtyard Charlotte SouthPark,NaN,...,False,False,False,False,False,False,False,False,False,False
919,072tu2_MFWmw-jb1O6DQYg,-k8gotDIdzMhyw8klSNGYQ,"Great stuff. They said they had limited menu selections, but there was a fixed menu option from which you could select an appetizer, entree and dessert. Everything was delicious. The portions were a little small, considering it was only 3 courses, I could have had more food; but they weren't unreasonably small. The pan sear scallop was delicious (but there was only the one) and the lamb was succulent with the extraordinary barolo reduction. The molten cake I had for dessert was a little too molten for my liking, but still tasty. I would recommend this place and appreciate all the Yelpers reviews that directed me to it in the first place. It's definitely worth a trip. FYI: it's tucked in a shopping center; very unassuming. Also worth noting -- I was seated next to several peo...",wINwaJCtSPsE3uJKUtzgLg,4,2010-07-20,review,"['Restaurants', 'Italian']",Fiamma,True,...,False,False,False,False,False,False,False,False,False,False
943,072tu2_MFWmw-jb1O6DQYg,NV7hA7yLbdjHftWCI9Nb5g,"I dined here as a second choice after the wait at a nearby spot was too long. Chuy's was fine, but not particularly memorable. My server was friendly, attentive and knowledgeable about the menu. I had some enchiladas that are one of the signature dishes due to my server's recommendation. That was an odd choice for me because I generally find enchiladas to be a gooey mess. These had some texture and I liked the chicken inside. I also ordered guacamole which was good, but not great. Restaurant guac is always overpriced, and unless you must have it, I would pass here, for the price. I thought the salsa with the chips was pretty runny. So overall, this was fine. I wouldn't go out of my way to visit again, but I wouldn't throw a fit if someone wanted to go here.",Iw_qZEDbHv9Jr4eC_2lfKA,3,2014-03-27,review,"['Tex-Mex', 'Restaurants']",Chuy's,True,...,False,False,False,False,False,False,False,False,False,False
1189,nEYPahVwXGD2Pjvgkm7QqQ,N0G8dlh70oamoDjUVgRvYQ,"I will never forget you delicious iced chai! So good. It's a fun and quirky coffee shop, that has delicious chai, what more can you ask for? Maybe a dirty chai, with a shot of espresso?? And pumpkin rolls in June? I'm in! Although I wasn't as into the roll as much as the chai, it's still a solid shop in my book.",RGmR2iBzqI3LEMDE6J8nJA,4,2014-07-15,review,"['Food', 'Coffee & Tea', 'Internet Cafes', 'Bagels']",Smelly Cat Coffee House,NaN,...,False,False,False,False,False,False,False,False,False,False
1260,nEYPahVwXGD2Pjvgkm7QqQ,oZFssH8Eci2eaOvCEndFcQ,"This place. I cannot stop talking about it. I want to go to there. All. The. Time! It's such an incredible time! Of course if I lived around town, I don't know if I could afford it! At $39 for a ""cool pass"" entrance it is not cheap. But was it worth it? TOTALLY! The Cool Pass gets you access to everything except for the White Water Rafting. So what can you do? Well, really, what CAN'T you do! We went zip lining over the white waters, kayaking in the river and went through the rope course. 

In [53]:
Charlotte.to_csv('yelp_charlotte_v1.csv', delimiter = '\t')